# Unificação de pesos do adaptador com os do modelo

### Imports

In [ ]:
from unsloth import FastVisionModel
from getpass import getpass

import torch

from scripts.authentication import authenticate_huggingface

### Autenticação

In [ ]:
authenticate_huggingface()

### Carregamento do modelo

In [ ]:
MODEL_NAME = '../weights/LLaMA_DERM_QLoRA_9500_11B'

model, tokenizer = FastVisionModel.from_pretrained(
    MODEL_NAME,
    load_in_4bit = True,
    use_gradient_checkpointing = 'unsloth',
)

### Salvamento da versão unificada do modelo

In [ ]:
model.save_pretrained_merged('model', tokenizer, save_method = 'merged_16bit')

### Envio do modelo para o Hugging Face

In [ ]:
hf_token = getpass('Enter your Hugging Face token: ')

model.push_to_hub_merged('ErFer7/LLaDerm', tokenizer, save_method='merged_16bit', token=hf_token)